# Classification Neural Network

## Dataset Prep

In [1]:
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()
X, y = iris.data, iris.target


In [3]:
n_features = X.shape[1]
n_features

4

In [4]:
from sklearn.preprocessing import StandardScaler
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
from sklearn.model_selection import train_test_split
# train-test slpit
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=0)

## Dataset Pytorch

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader

class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)  # 1D labels for CrossEntropyLoss

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.X[index], self.y[index]


In [7]:
train_dataset = IrisDataset(X=X_train, y=y_train)
test_dataset = IrisDataset(X=X_test, y=y_test)

In [8]:
# dataloader
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

## Neural Network Model

In [9]:
import torch.nn as nn

class IrisModel(nn.Module):
    def __init__(self):
        super(IrisModel, self).__init__()
    
        self.model = nn.Sequential(
            nn.Linear(in_features=4, out_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=3) # 3 output classes
        )

    def forward(self, x):
        return self.model(x)

## Training

In [10]:
# Training config
config = {
    'batch_size': 32,
    'epochs': 100,
    'learning_rate': 1e-4,
}

In [12]:
# training
model = IrisModel()
# loss
criterion = nn.CrossEntropyLoss()
#Optimizer
lr = config['learning_rate']
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# epochs
epochs = config['epochs']

In [13]:
# training loop
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 10/100, Loss: 1.0689
Epoch 20/100, Loss: 1.0089
Epoch 30/100, Loss: 0.9471
Epoch 40/100, Loss: 0.8828
Epoch 50/100, Loss: 0.8213
Epoch 60/100, Loss: 0.7648
Epoch 70/100, Loss: 0.7124
Epoch 80/100, Loss: 0.6639
Epoch 90/100, Loss: 0.6196
Epoch 100/100, Loss: 0.5790
